<a href="https://colab.research.google.com/github/alisonsoong/NASA-SEES-Internship-2021/blob/main/WildfirePractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Created by Alison Soong


Algorithm ideas:
- https://www.ci.richmond.ca.us/DocumentCenter/View/53610/City-of-Richmond-Wildfire-Preparedness-and-Evacuation-Guide "A home within 10 miles of a wildfire will most likely be effected by wind driven embers which can be a risk to your property" --> might be a good buffer area to look at, but would need to convert burn scar area into a geometry (?)


Datasets:
- https://developers.google.com/earth-engine/datasets/catalog/WorldPop_GP_100m_pop#bands
- https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD64A1

References:
- https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/notebooks/00_geemap_key_features.ipynb#scrollTo=zduDFJXjyeiz

# Import libraries

In [ ]:
import ee # Import Earth Engine API

In [ ]:
## Trigger the authentication flow. You only need to do this once
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=ltSgWN2MFvjusR1Rhs5agIlBzF8FV0k8k7raU06uJgU&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhULaq-Aci1Cu7FgXVzpFsuCGeHgUtXjI3vT8aOvGdGLZAE5C8O1TY

Successfully saved authorization token.


In [ ]:
# Installs geemap package
import subprocess

try:
    import geemap
    print("geemap is imported and ready to use in Colab")
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap
    print("geemap is now installed, imported and ready to use in Colab") 


geemap is imported and ready to use in Colab


In [ ]:
try:
    import ee
    import seaborn as sns
    import matplotlib.pyplot as plt
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap seaborn matplotlib
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
        !conda install seaborn matplotlib -y
    import ee
    import seaborn as sns
    import matplotlib.pyplot as plt

# Looking at populations and burn scars (burn areas)


Here is some information found about using gt an lt to specify a range, like you ask.

```
# Only keep pixels where population is greater than 0
# This might not work on your dataset.
populated2020 = pop2020.updateMask(pop2020.gt(0.0))
```

If you need to define a range instead, you could do this 
```
onlyWater = classified.updateMask(classified.gt(1).and(classified.lt(3))
```

eq() - equals
neq() - not equals
lt() - less than 
gt() - greater than
lte() - less than or equals
gte() - greater than or equals
and() - logical and
or() - logical or

In [ ]:

import os
california_area = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME', 'California'))



In [ ]:
# population grid
popdataset2020 = ee.ImageCollection("WorldPop/GP/100m/pop")\
    .filterDate("2020-01-01", "2021-01-01")

pop2020 = popdataset2020.mean().clip(california_area)

pop_params = {
  'bands': ['population'],
  'min': 1.0,
  'max': 60.0,
  'palette' : ['bdd7e7','6baed6','2171b5']
}


In [ ]:
# burned
burnedCollection2020 = ee.ImageCollection('MODIS/006/MCD64A1').filter(ee.Filter.date('2020-01-01', '2021-01-01'))

burned2020 = burnedCollection2020.select('BurnDate').mean().clip(california_area)

burned_params = {
  'min': 30.0,
  'max': 341.0,
  'palette':['red']
}

In [ ]:
# reduce population grid to only burned areas, and add values
def mask(image):
  empty = 0
  qa = image.select('population');
  popMask = qa.neq(0) # masking out areas that have 0 population
  burnedMask = burned2020.select('BurnDate').neq(0) # masking out areas that were not burned
  mask = popMask.And(burnedMask) # only keep areas that were both burned and have a population
  return image.updateMask(mask).select("population").copyProperties(image, ['population'])

burnedpop2020 = popdataset2020.map(mask)

impacted_pop_params = {
  'bands': ['population'],
  'min': 0.0,
  'max': 60.0,
  'palette' : ['white','bdd7e7','6baed6','2171b5']
}

# not exactly sure what's going wrong here?
popSum = burnedpop2020.reduce(ee.Reducer.sum())

sumDictionary = popSum.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': california_area,
  'scale': 100,
  'maxPixels': 1e12
})

print('Population affected in 2020:', sumDictionary.getInfo()['population_sum'])

totalPopSum = pop2020.reduce(ee.Reducer.sum())

sumTotalDictionary = totalPopSum.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': california_area,
  'scale': 100,
  'maxPixels': 1e12
})

print('Total population in California 2020:', sumTotalDictionary.getInfo()['sum'])

print('Percent of population affected by wildfires in 2020:\n ', sumDictionary.getInfo()['population_sum']/sumTotalDictionary.getInfo()['sum']*100, '%', sep='')


Population affected in 2020: 28111.26220767843
Total population in California 2020: 35521219.45021494
Percent of population affected by wildfires in 2020:
 0.07913935006391884%


Something I might look more into: edge detection -> https://developers.google.com/earth-engine/guides/image_edges

In [ ]:
# Map it
roi = ee.Geometry.Point(-122.438005, 37.729844) # (long, lat)

Map = geemap.Map()
Map.add_basemap("SATELLITE")

Map.addLayer(pop2020, pop_params, "Population 2020", True)
Map.addLayer(burned2020, burned_params, 'Burned Area 2020', True, 0.6)
Map.addLayer(burnedpop2020, impacted_pop_params, "Impacted Population 2020", True)
roi = ee.Geometry.Point(-122.438005, 37.729844) # (long, lat)

# legend for population
legend_dict = {
    'No population': '000000',
    'Low population':'6baed6',
    'High population': '2171b5',
    'Area with population affected by wildfires': 'ffffff',
    'Burn areas': 'ff0000'
}

Map.add_legend(legend_title="Population and Burn Scars", legend_dict=legend_dict)

Map.centerObject(roi, 6)
Map.addLayerControl()
Map